# Zero Noise Extrapolation (ZNE) for indirect-control system
---

In [5]:
# Importing modules
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from typing import List, Tuple
from collections import Counter
import itertools
import scipy.linalg as la
import random
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline


In [77]:
def get_monomials(n: int, d: int) -> list[str]:
    """Compute monomials of degree `d` in graded lexicographical order."""
    variables = [f"λ_{i}" for i in range(1, n + 1)]
    
    monomials = []
    for degree in range(d, -1, -1):
        # Generate combinations for the current degree
        combos = list(itertools.combinations_with_replacement(variables, degree))
        
        # Sort combinations lexicographically
        combos.sort()
        
        # Construct monomials from sorted combinations
        for combo in combos:
            monomial_parts = []
            counts = Counter(combo)
            # Ensure variables are processed in lexicographical order
            for var in sorted(counts.keys()):
                count = counts[var]
                if count > 1:
                    monomial_parts.append(f"{var}**{count}")
                else:
                    monomial_parts.append(var)
            monomial = "*".join(monomial_parts)
            # Handle the case where degree is 0
            if not monomial:
                monomial = "1"
            monomials.append(monomial)
    # "1" should be the first monomial. Note that order d > c > b > a means vector of monomials = [a, b, c, d].            
    return monomials[::-1]

def sample_matrix(sample_points: list[int], degree: int) -> np.ndarray:
    """Construct a matrix from monomials evaluated at sample points."""
    n = len(sample_points[0])  # Number of variables based on the first sample point
    monomials = get_monomials(n, degree)
    matrix = np.zeros((len(sample_points), len(monomials)))

    for i, point in enumerate(sample_points):
        for j, monomial in enumerate(monomials):
            var_mapping = {f"λ_{k+1}": point[k] for k in range(n)}
            matrix[i, j] = eval(monomial, {}, var_mapping)
    return matrix

def get_eta_coeffs_from_sample_matrix(mat: np.ndarray) -> list[float]:
    """Given a sample matrix compute the eta coefficients."""
    n_rows, n_cols = mat.shape
    if n_rows != n_cols:
        raise ValueError("The matrix must be square.")

    det_m = np.linalg.det(mat)    
    if det_m == 0:
        raise ValueError("The matrix is singular.")
    
    terms = []
    for i in range(n_rows):
        new_mat = mat.copy()
        # new_mat[i] = np.array([[0] * (n_cols - 1) + [1]])   
        new_mat[i] = np.array([[1] * (n_cols - 1) + [0]])       
        terms.append(np.linalg.det(new_mat) / det_m)

    return terms

In [91]:
data_Q7N1 = [
   (4, 1, 0, -4.8169220971038795), # Noise factor [0, 0, 0]
    (12, 1, 0, -4.2189237350915185), # Noise factor [1, 0, 0]
    (12, 3, 4, -2.9070960640998686), # Noise factor [1, 1, 0]
    (12, 3, 12, -2.253565700341366), # Noise factor [1, 1, 1]

    (20, 1, 0, -3.701640647812517), # Noise factor [2, 0, 0]
    (20, 3, 4, -2.5556168431723947), # Noise factor [2, 1, 0]
    (20, 3, 12, -1.9834895523974967), # Noise factor [2, 1, 1]
    (20, 5, 4, -1.7784106506781265), # Noise factor [2, 2, 0]
    (20, 5, 12, -1.086614154166062), # Noise factor [2, 2, 1]
    (20, 5, 20, -0.6731855339471752), # Noise factor [2, 2, 2]

    (28, 1, 0, -3.2530466718309436), # Noise factor [3, 0, 0]
    (28, 3, 4, -2.249816877366662), # Noise factor [3, 1, 0]
    (28, 3, 12, -1.7479790647310043), # Noise factor [3, 1, 1]
    (28, 5, 4, -1.56801564275309), # Noise factor [3, 2, 0]
    (28, 5, 12, -0.9596543348762642), # Noise factor [3, 2, 1]
    (28, 5, 20, -0.5952369595178605), # Noise factor [3, 2, 2]
    (28, 7, 4, -1.1009119771157778), # Noise factor [3, 3, 0]
    (28, 7, 12, -0.5372973838819411), # Noise factor [3, 3, 1]
    (28, 7, 20, -0.2694210227511671), # Noise factor [3, 3, 2]
    (28, 7, 28, -0.1382972369948969), # Noise factor [3, 3, 3]


    (36, 1, 0, -2.8630374748518133), # Noise factor [4, 0, 0]
    (36, 3, 4, -1.9831008116380482), # Noise factor [4, 1, 0]
    (36, 3, 12, -1.5421097437540896), # Noise factor [4, 1, 1]
    (36, 5, 4, -1.3839398211641174), # Noise factor [4, 2, 0]
    (36, 5, 12, -0.848102920086503), # Noise factor [4, 2, 1]
    (36, 5, 20, -0.5264734057146431), # Noise factor [4, 2, 2]
    (36, 7, 4, -0.9727444036453345), # Noise factor [4, 3, 0]
    (36, 7, 12, -0.4753642005213387), # Noise factor [4, 3, 1]
    (36, 7, 20, -0.23842325519270866), # Noise factor [4, 3, 2]
    (36, 7, 28, -0.12229419340336584), # Noise factor [4, 3, 3]
    (36, 9, 4, -0.6883999795236337), # Noise factor [4, 4, 0]
    (36, 9, 12, -0.27105917176707967), # Noise factor [4, 4, 1]
    (36, 9, 20, -0.11122418284241704), # Noise factor [4, 4, 2]
    (36, 9, 28, -0.047198327335706176), # Noise factor [4, 4, 3]
    (36, 9, 36, -0.020583131129675067), # Noise factor [4, 4, 4]
    (44, 1, 0, -2.5231063283099395), # Noise factor [5, 0, 0]
    (44, 3, 4, -1.7498989194396781), # Noise factor [5, 1, 0]
    (44, 3, 12, -1.3617137677305078), # Noise factor [5, 1, 1]
    (44, 5, 4, -1.2225024244022056), # Noise factor [5, 2, 0]
    (44, 5, 12, -0.7498619287072883), # Noise factor [5, 2, 1]
    (44, 5, 20, -0.46567706790716373), # Noise factor [5, 2, 2]
    (44, 7, 4, -0.8600076108241173), # Noise factor [5, 3, 0]
    (44, 7, 12, -0.4205644718415898), # Noise factor [5, 3, 1]
    (44, 7, 20, -0.21084934645317444), # Noise factor [5, 3, 2]
    (44, 7, 28, -0.10799141130577888), # Noise factor [5, 3, 3]
    (44, 9, 4, -0.6090104996217656), # Noise factor [5, 4, 0]
    (44, 9, 12, -0.23981140378588708), # Noise factor [5, 4, 1]
    (44, 9, 20, -0.09822156028796472), # Noise factor [5, 4, 2]
    (44, 9, 28, -0.04153225954048584), # Noise factor [5, 4, 3]
    (44, 9, 36, -0.018019315155465957), # Noise factory [5, 4, 4]
    (44, 11, 4, -0.4339807857888248), # Noise factor [5, 5, 0]
    (44, 11, 12, -0.13878995911761374), # Noise factor [5, 5, 1]
    (44, 11, 20, -0.04687070760572831), # Nois factor [5, 5, 2]
    (44, 11, 28, -0.01650190914701817), # Noise factor [5, 5, 3]
    (44, 11, 36, -0.005998551538671056), # Noise factor [5, 5, 4]
    (44, 11, 44, -0.0022352160842883627), # Noise factor [5, 5, 5]
]
data_Q7N2 = [
    (4, 1, 0, -5.8440279326372915), # Noise factor [0, 0, 0]
    (12, 1, 0, -5.767710734005716), # Noise factor [1, 0, 0]
    (12, 3, 4, -5.5574505982244204), # Noise factor [1, 1, 0]
    (12, 3, 12, -5.415680923506571), # Noise factor [1, 1, 1]
    (20, 1, 0, -5.692515294378422), # Noise factor [2, 0, 0]
    (20, 3, 4, -5.485126624256428), # Noise factor [2, 1, 0]
    (20, 3, 12, -5.345296450663245), # Noise factor [2, 1, 1]
    (20, 5, 4, -5.28572626703126), # Noise factor [2, 2, 0]
    (20, 5, 12, -5.0204755465486235), # Noise factor [2, 2, 1]
    (20, 5, 20, -4.7693601557302925), # Noise factor [2, 2, 2]
    (28, 1, 0, -5.618423048764279), # Noise factor [3, 0, 0]
    (28, 3, 4, -5.413860969567599), # Noise factor [3, 1, 0]
    (28, 3, 12, -5.2759399546339845), # Noise factor [3, 1, 1]
    (28, 5, 4, -5.217172191111792), # Noise factor [3, 2, 0]
    (28, 5, 12, -4.955531868302668), # Noise factor [3, 2, 1]
    (28, 5, 20, -4.707823064601974), # Noise factor [3, 2, 2]
    (28, 7, 4, -5.02803624143284), # Noise factor [3, 3, 0]
    (28, 7, 12, -4.6556926164174985), # Noise factor [3, 3, 1]
    (28, 7, 20, -4.312567974796216), # Noise factor [3, 3, 2]
    (28, 7, 28, -3.996227428279443), # Noise factor [3, 3, 3]
    (36, 1, 0, -5.545415766512828), # Noise factor [4, 0, 0]
    (36, 3, 4, -5.343636187100848), # Noise factor [4, 1, 0]
    (36, 3, 12, -5.207594518316608), # Noise factor [4, 1, 1]
    (36, 5, 4, -5.149616800894365), # Noise factor [4, 2, 0]
    (36, 5, 12, -4.891530698757975), # Noise factor [4, 2, 1]
    (36, 5, 20, -4.647175681186043), # Noise factor [4, 2, 2]
    (36, 7, 4, -4.963042019784506), # Noise factor [4, 3, 0]
    (36, 7, 12, -4.595741134191412), # Noise factor [4, 3, 1]
    (36, 7, 20, -4.2572407149814575), # Noise factor [4, 3, 2]
    (36, 7, 28, -3.9451429304740966), # Noise factor [4, 3, 3]
    (36, 9, 4, -4.783609813039699), # Noise factor [4, 4, 0]
    (36, 9, 12, -4.318849596181545), # Noise factor [4, 4, 1]
    (36, 9, 20, -3.9018408193870258), # Noise factor [4, 4, 2]
    (36, 9, 28, -3.5273844220317856), # Noise factor [4, 4, 3]
    (36, 9, 36, -3.19088123921418), # Noise factor [4, 4, 4]
    (44, 1, 0, -5.47347554489224), # Noise factor [5, 0, 0]
    (44, 3, 4, -5.274435142917494), # Noise factor [5, 1, 0]
    (44, 3, 12, -5.1402435277181615), # Noise factor [5, 1, 1]
    (44, 5, 4, -5.0830436963297565), # Noise factor [5, 2, 0]
    (44, 5, 12, -4.828456614514787), # Noise factor [5, 2, 1]
    (44, 5, 20, -4.587403496885676), # Noise factor [5, 2, 2]
    (44, 7, 4, -4.8989904156003945), # Noise factor [5, 3, 0]
    (44, 7, 12, -4.536654229184054), # Noise factor [5, 3, 1]
    (44, 7, 20, -4.202706881876177), # Noise factor [5, 3, 2]
    (44, 7, 28, -3.894786952884057), # Noise factor [5, 3, 3]
    (44, 9, 4, -4.721977849857405), # Noise factor [5, 4, 0]
    (44, 9, 12, -4.263481011315959), # Noise factor [5, 4, 1]
    (44, 9, 20, -3.8520561034006797), # Noise factor [5, 4, 2]
    (44, 9, 28, -3.4825825685340424), # Noise factor [5, 4, 3]
    (44, 9, 36, -3.1505301287653014), # Noise factor [5, 4, 4]
    (44, 11, 4, -4.551721283711219), # Noise factor [5, 5, 0]
    (44, 11, 12, -4.007678819175834), # Noise factor [5, 5, 1]
    (44, 11, 20, -3.5322660924853015), # Noise factor [5, 5, 2]
    (44, 11, 28, -3.116327477599458), # Noise factor [5, 5, 3]
    (44, 11, 36, -2.751999280260503), # Noise factor [5, 5, 4]
    (44, 11, 44, -2.432517271240873), # Noise factor [5, 5, 5]
]

noise_level = [(nR, nT, nY) for nR, nT, nY, E in data_Q7N2]
energies = [E for nR, nT, nY, E in data_Q7N1]

In [92]:
len(data_Q7N1)

56

In [93]:
n, d = 3, 5
get_monomials(n, d)

['1',
 'λ_3',
 'λ_2',
 'λ_1',
 'λ_3**2',
 'λ_2*λ_3',
 'λ_2**2',
 'λ_1*λ_3',
 'λ_1*λ_2',
 'λ_1**2',
 'λ_3**3',
 'λ_2*λ_3**2',
 'λ_2**2*λ_3',
 'λ_2**3',
 'λ_1*λ_3**2',
 'λ_1*λ_2*λ_3',
 'λ_1*λ_2**2',
 'λ_1**2*λ_3',
 'λ_1**2*λ_2',
 'λ_1**3',
 'λ_3**4',
 'λ_2*λ_3**3',
 'λ_2**2*λ_3**2',
 'λ_2**3*λ_3',
 'λ_2**4',
 'λ_1*λ_3**3',
 'λ_1*λ_2*λ_3**2',
 'λ_1*λ_2**2*λ_3',
 'λ_1*λ_2**3',
 'λ_1**2*λ_3**2',
 'λ_1**2*λ_2*λ_3',
 'λ_1**2*λ_2**2',
 'λ_1**3*λ_3',
 'λ_1**3*λ_2',
 'λ_1**4',
 'λ_3**5',
 'λ_2*λ_3**4',
 'λ_2**2*λ_3**3',
 'λ_2**3*λ_3**2',
 'λ_2**4*λ_3',
 'λ_2**5',
 'λ_1*λ_3**4',
 'λ_1*λ_2*λ_3**3',
 'λ_1*λ_2**2*λ_3**2',
 'λ_1*λ_2**3*λ_3',
 'λ_1*λ_2**4',
 'λ_1**2*λ_3**3',
 'λ_1**2*λ_2*λ_3**2',
 'λ_1**2*λ_2**2*λ_3',
 'λ_1**2*λ_2**3',
 'λ_1**3*λ_3**2',
 'λ_1**3*λ_2*λ_3',
 'λ_1**3*λ_2**2',
 'λ_1**4*λ_3',
 'λ_1**4*λ_2',
 'λ_1**5']

In [94]:
mat = sample_matrix(noise_level, d)
eta = get_eta_coeffs_from_sample_matrix(mat)
eta

[2.159809366865385,
 -2.9451955159507137,
 -0.9817314147951062,
 0.9817314147951062,
 3.4102274576818528,
 0.6200408935546056,
 -0.6200408935545703,
 0.10334014892603785,
 1.4467620849610505,
 -1.5501022338860526,
 -2.3997904459630015,
 -0.37891387939468485,
 0.3789138793948572,
 -0.034446716308172226,
 -0.482254028320132,
 0.5167007446285973,
 0.011482238769049411,
 -0.21816253662138418,
 -1.067848205566347,
 1.2745285034181315,
 0.9256337483723797,
 0.13713073730477637,
 -0.13713073730483094,
 0.010826110839906936,
 0.1515655517577209,
 -0.16239166259753102,
 -0.001968383789168381,
 0.03739929199217039,
 0.18305969238279748,
 -0.2184906005858354,
 -0.013942718505682733,
 0.040023803710810414,
 0.08956146240257434,
 0.42845153808583014,
 -0.5440940856929928,
 -0.1506846110027368,
 -0.021526336670016518,
 0.021526336670053228,
 -0.0015945434569924113,
 -0.0223236083984111,
 0.023918151855305953,
 0.00025177001938743134,
 -0.004783630371134392,
 -0.02341461181639835,
 0.0279464721679723

In [66]:
zne = 0
for i, n in enumerate(eta):
    # print(n)
    # print(energies[i])
    zne += energies[i]*n
print(zne)

0.3204195495953138


In [95]:
np.dot(energies, eta)

-5.025450266489713

In [57]:
(-0.1249999999999999*-4.8169220971038795)+(0.12499999999999997*-2.253565700341366)

0.3204195495953138

In [90]:
import itertools

# Provided data
data = [
    (4, 1, 0, -4.8169220971038795), 
    (12, 1, 0, -4.2189237350915185), 
    (12, 3, 12, -2.253565700341366), 
    (20, 1, 0, -3.701640647812517), 
    (20, 3, 4, -2.5556168431723947), 
    (20, 3, 12, -1.9834895523974967), 
    (20, 5, 4, -1.7784106506781265), 
    (20, 5, 12, -1.086614154166062), 
    (20, 5, 20, -0.6731855339471752), 
    (28, 1, 0, -3.2530466718309436), 
    (28, 3, 4, -2.249816877366662), 
    (28, 3, 12, -1.7479790647310043), 
    (28, 5, 4, -1.56801564275309), 
    (28, 5, 12, -0.9596543348762642), 
    (28, 5, 20, -0.5952369595178605), 
    (28, 7, 4, -1.1009119771157778), 
    (28, 7, 12, -0.5372973838819411), 
    (28, 7, 20, -0.2694210227511671), 
    (28, 7, 28, -0.1382972369948969), 
    (36, 1, 0, -2.8630374748518133), 
    (36, 3, 4, -1.9831008116380482), 
    (36, 3, 12, -1.5421097437540896), 
    (36, 5, 4, -1.3839398211641174), 
    (36, 5, 12, -0.848102920086503), 
    (36, 5, 20, -0.5264734057146431), 
    (36, 7, 4, -0.9727444036453345), 
    (36, 7, 12, -0.4753642005213387), 
    (36, 7, 20, -0.23842325519270866), 
    (36, 7, 28, -0.12229419340336584), 
    (36, 9, 4, -0.6883999795236337), 
    (36, 9, 12, -0.27105917176707967), 
    (36, 9, 20, -0.11122418284241704), 
    (36, 9, 28, -0.047198327335706176), 
    (36, 9, 36, -0.020583131129675067), 
    (44, 1, 0, -2.5231063283099395), 
    (44, 3, 4, -1.7498989194396781), 
    (44, 3, 12, -1.3617137677305078), 
    (44, 5, 4, -1.2225024244022056), 
    (44, 5, 12, -0.7498619287072883), 
    (44, 5, 20, -0.46567706790716373), 
    (44, 7, 4, -0.8600076108241173), 
    (44, 7, 12, -0.4205644718415898), 
    (44, 7, 20, -0.21084934645317444), 
    (44, 7, 28, -0.10799141130577888), 
    (44, 9, 4, -0.6090104996217656), 
    (44, 9, 12, -0.23981140378588708), 
    (44, 9, 20, -0.09822156028796472), 
    (44, 9, 28, -0.04153225954048584), 
    (44, 9, 36, -0.018019315155465957), 
    (44, 11, 4, -0.4339807857888248), 
    (44, 11, 12, -0.13878995911761374), 
    (44, 11, 20, -0.04687070760572831), 
    (44, 11, 28, -0.01650190914701817), 
    (44, 11, 36, -0.005998551538671056), 
    (44, 11, 44, -0.0022352160842883627),
]

# Generate all possible combinations of noise factors
max_noise_factors = [5, 5, 5]
all_combinations = list(itertools.product(range(max_noise_factors[0] + 1), range(max_noise_factors[1] + 1), range(max_noise_factors[2] + 1)))

# Convert the data to a set of tuples for easy comparison
data_set = set((x, y, z) for x, y, z, E in data)

# Find the missing combination
missing_combination = None
for combination in all_combinations:
    if combination not in data_set:
        missing_combination = combination
        break

print("Missing noise factor combination:", missing_combination)


Missing noise factor combination: (0, 0, 0)
